In [1]:
import pandas as pd
import numpy as np
import string
import re
import random
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer


In [2]:
df=pd.read_csv(r"D:\Capstone\Datasets\Data_preparation\preprocessed_labels3_data.csv")


In [3]:
df.head(5)

,docid,text,topics,count_topics,text_expanded,text_wo_punct,remove_titles,text_tokenized,text_wo_stopwords,text_lemmatized,processed_text
0,2289,CompuServe Corp. Tuesday reported a surprising...,"['PERFORMANCE', 'ACCOUNTS/EARNINGS', 'CORPORAT...",3,CompuServe Corp. Tuesday reported a surprising...,compuserve corp tuesday reported a surprisingl...,compuserve corp tuesday reported a surprisingl...,"['compuserve', 'corp', 'tuesday', 'reported', ...","['compuserve', 'corp', 'tuesday', 'reported', ...","['compuserve', 'corp', 'tuesday', 'report', 's...",compuserve corp tuesday report surprisingly la...
1,2290,If dining at Planet Hollywood made you feel li...,"['STRATEGY/PLANS', 'NEW_PRODUCTS/SERVICES', 'C...",3,If dining at Planet Hollywood made you feel li...,if dining at planet hollywood made you feel li...,if dining at planet hollywood made you feel li...,"['if', 'dining', 'at', 'planet', 'hollywood', ...","['dining', 'planet', 'hollywood', 'made', 'fee...","['din', 'planet', 'hollywood', 'make', 'feel',...",din planet hollywood make feel like movie star...
2,2302,Columbia/HCA Healthcare Corp.'s proposed $299....,"['OWNERSHIP_CHANGES', 'MERGERS/ACQUISITIONS', ...",3,Columbia/HCA Healthcare Corp.'s proposed $299....,columbiahca healthcare corps proposed million...,columbiahca healthcare corps proposed million...,"['columbiahca', 'healthcare', 'corps', 'propos...","['columbiahca', 'healthcare', 'corps', 'propos...","['columbiahca', 'healthcare', 'corps', 'propos...",columbiahca healthcare corps propose million a...
3,2307,World oil prices slipped on Tuesday in a marke...,"['COMMODITY_MARKETS', 'ENERGY_MARKETS', 'MARKE...",3,World oil prices slipped on Tuesday in a marke...,world oil prices slipped on tuesday in a marke...,world oil prices slipped on tuesday in a marke...,"['world', 'oil', 'prices', 'slipped', 'on', 't...","['world', 'oil', 'prices', 'slipped', 'tuesday...","['world', 'oil', 'price', 'slip', 'tuesday', '...",world oil price slip tuesday market refiners s...
4,2312,Gulf War hero Colin Powell lent his prestige a...,"['GOVERNMENT/SOCIAL', 'DOMESTIC_POLITICS', 'EL...",3,Gulf War hero Colin Powell lent his prestige a...,gulf war hero colin powell lent his prestige a...,gulf war hero colin powell lent his prestige a...,"['gulf', 'war', 'hero', 'colin', 'powell', 'le...","['gulf', 'war', 'hero', 'colin', 'powell', 'le...","['gulf', 'war', 'hero', 'colin', 'powell', 'le...",gulf war hero colin powell lend prestige popul...


In [4]:
df['topics'].value_counts().head(10)

topics
['PERFORMANCE', 'COMMENT/FORECASTS', 'CORPORATE/INDUSTRIAL']             53939
['PERFORMANCE', 'ACCOUNTS/EARNINGS', 'CORPORATE/INDUSTRIAL']             50740
['COMMODITY_MARKETS', 'SOFT_COMMODITIES', 'MARKETS']                     36090
['OWNERSHIP_CHANGES', 'MERGERS/ACQUISITIONS', 'CORPORATE/INDUSTRIAL']    23106
['MONEY_MARKETS', 'INTERBANK_MARKETS', 'MARKETS']                        20439
['GOVERNMENT_FINANCE', 'GOVERNMENT_BORROWING', 'ECONOMICS']              17671
['COMMODITY_MARKETS', 'ENERGY_MARKETS', 'MARKETS']                       17271
['MONEY_MARKETS', 'FOREX_MARKETS', 'MARKETS']                            15811
['FUNDING/CAPITAL', 'BONDS/DEBT_ISSUES', 'CORPORATE/INDUSTRIAL']          9489
['COMMODITY_MARKETS', 'METALS_TRADING', 'MARKETS']                        7750
Name: count, dtype: int64

In [5]:

top_52_topics = df['topics'].value_counts().head(52)

# Create an empty DataFrame to store sampled data
sampled_data = pd.DataFrame(columns=df.columns)

for topic, count in top_52_topics.items():
    # Filter rows for the current topic
    topic_data = df[df['topics'] == topic]
    
    # If there are more than 500 samples, randomly select 500
    if count > 500:
        topic_data = topic_data.sample(n=500, random_state=42)
    
    # Append selected samples to the sampled_data DataFrame
    sampled_data = pd.concat([sampled_data, topic_data])

# Reset index of the sampled data
sampled_data.reset_index(drop=True, inplace=True)

# Print the shape of the sampled data
print("Shape of sampled data:", sampled_data.shape)


Shape of sampled data: (26000, 11)


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorizer
vect = CountVectorizer()  
vects = vect.fit_transform(sampled_data.processed_text)
vects.shape

(26000, 71994)

In [7]:
tdm = vects.T

term_document_matrix = pd.DataFrame.sparse.from_spmatrix(tdm, index=vect.get_feature_names_out(), columns=sampled_data['docid'].astype(str))

print(term_document_matrix.shape)

(71994, 26000)


## Max Normalization

In [8]:
import numpy as np

max_values = term_document_matrix.max(axis=0)
normalized_term_document_matrix =np.divide(term_document_matrix, max_values)
normalized_term_document_matrix=normalized_term_document_matrix.values


## Truncate_SVD

In [9]:
from sklearn.decomposition import TruncatedSVD
import numpy as np

k = 50 
svd = TruncatedSVD(n_components=k)

U = svd.fit_transform(normalized_term_document_matrix)
S = np.diag(svd.singular_values_)  
VT = svd.components_


S_inv = np.linalg.inv(S)

print("Inverse of S:")
print(S_inv)

Inverse of S:
[[0.00720928 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0157339  0.         ... 0.         0.         0.        ]
 [0.         0.         0.0207617  ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.05207254 0.         0.        ]
 [0.         0.         0.         ... 0.         0.05246189 0.        ]
 [0.         0.         0.         ... 0.         0.         0.05318803]]


In [10]:
Xq_prime = normalized_term_document_matrix.T

print("Xq_prime shape: ",Xq_prime.shape)
print("U shape: ",U.shape)
print("S_inv shape: ",S_inv.shape)


Xq_prime shape:  (26000, 71994)
U shape:  (71994, 50)
S_inv shape:  (50, 50)


In [11]:
Dq = Xq_prime.dot(U).dot(S_inv)
Dq.shape

(26000, 50)

In [12]:
sampled_data = sampled_data.reset_index(drop=True)
sampled_data.shape

(26000, 11)

In [13]:
sampled_data['topics'][9]

"['PERFORMANCE', 'COMMENT/FORECASTS', 'CORPORATE/INDUSTRIAL']"

In [14]:
import ast

# Convert string representation of list to actual list
sampled_data['topics'] = sampled_data['topics'].apply(ast.literal_eval)

# Now the 'topics' column will contain lists instead of strings
print(sampled_data['topics'][9])  # Print the value at index 9 to verify


['PERFORMANCE', 'COMMENT/FORECASTS', 'CORPORATE/INDUSTRIAL']


In [15]:
print(sampled_data['topics'][0]) 

['PERFORMANCE', 'COMMENT/FORECASTS', 'CORPORATE/INDUSTRIAL']


In [16]:
from sklearn.preprocessing import MultiLabelBinarizer

# Instantiate the binarizer
mlb = MultiLabelBinarizer()

# Fit and transform the 'topics' column of the DataFrame
binary_labels = mlb.fit_transform(sampled_data['topics'])

# Create a new DataFrame from the binary labels with appropriate column names
mlb_df = pd.DataFrame(binary_labels, columns=mlb.classes_, index=sampled_data.index)
mlb_df.head(3)


,ACCOUNTS/EARNINGS,ASSET_TRANSFERS,BALANCE_OF_PAYMENTS,"BIOGRAPHIES,_PERSONALITIES,_PEOPLE",BONDS/DEBT_ISSUES,CAPACITY/FACILITIES,COMMENT/FORECASTS,COMMODITY_MARKETS,CONSUMER_FINANCE,CONSUMER_PRICES,...,REGULATION/POLICY,RESERVES,RETAIL_SALES,SHARE_CAPITAL,SOFT_COMMODITIES,STRATEGY/PLANS,TRADE/RESERVES,"WAR,_CIVIL_WAR",WEATHER,WHOLESALE_PRICES
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
mlb.classes_

array(['ACCOUNTS/EARNINGS', 'ASSET_TRANSFERS', 'BALANCE_OF_PAYMENTS',
       'BIOGRAPHIES,_PERSONALITIES,_PEOPLE', 'BONDS/DEBT_ISSUES',
       'CAPACITY/FACILITIES', 'COMMENT/FORECASTS', 'COMMODITY_MARKETS',
       'CONSUMER_FINANCE', 'CONSUMER_PRICES', 'CONTRACTS/ORDERS',
       'CORPORATE/INDUSTRIAL', 'CREDIT_RATINGS', 'CRIME,_LAW_ENFORCEMENT',
       'DEFENCE', 'DISASTERS_AND_ACCIDENTS', 'DOMESTIC_POLITICS',
       'ECONOMICS', 'ECONOMIC_PERFORMANCE', 'ELECTIONS', 'ENERGY_MARKETS',
       'ENVIRONMENT_AND_NATURAL_WORLD', 'EUROPEAN_COMMUNITY',
       'EXPENDITURE/REVENUE', 'EXTERNAL_MARKETS', 'FOREX_MARKETS',
       'FUNDING/CAPITAL', 'GOVERNMENT/SOCIAL', 'GOVERNMENT_BORROWING',
       'GOVERNMENT_FINANCE', 'INDUSTRIAL_PRODUCTION', 'INFLATION/PRICES',
       'INTERBANK_MARKETS', 'INTERNATIONAL_RELATIONS', 'LOANS/CREDITS',
       'MANAGEMENT', 'MANAGEMENT_MOVES', 'MARKETS', 'MARKETS/MARKETING',
       'MERCHANDISE_TRADE', 'MERGERS/ACQUISITIONS', 'METALS_TRADING',
       'MONETARY/ECON

In [21]:
Dq.shape

(26000, 50)

In [22]:
mlb_df.shape

(26000, 61)

In [23]:
mlb_df

,ACCOUNTS/EARNINGS,ASSET_TRANSFERS,BALANCE_OF_PAYMENTS,"BIOGRAPHIES,_PERSONALITIES,_PEOPLE",BONDS/DEBT_ISSUES,CAPACITY/FACILITIES,COMMENT/FORECASTS,COMMODITY_MARKETS,CONSUMER_FINANCE,CONSUMER_PRICES,...,REGULATION/POLICY,RESERVES,RETAIL_SALES,SHARE_CAPITAL,SOFT_COMMODITIES,STRATEGY/PLANS,TRADE/RESERVES,"WAR,_CIVIL_WAR",WEATHER,WHOLESALE_PRICES
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
pip install iterative-stratification


Note: you may need to restart the kernel to use updated packages.


In [24]:
X_train, X_test, y_train, y_test = train_test_split(Dq, mlb_df, test_size=0.2, random_state=42)

num_classes = y_train.shape[1]
num_classes 

61

In [26]:
print("X_train: ",X_train.shape, '\t\t',"X_test: ",X_test.shape, '\n', "y_train: ",y_train.shape, '\t\t', "y_test: ",y_test.shape)

X_train:  (20800, 50) 		 X_test:  (5200, 50) 
 y_train:  (20800, 61) 		 y_test:  (5200, 61)


In [27]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(Dq, mlb_df, test_size=0.2, random_state=42)

num_classes = y_train.shape[1]

# Define the model architecture without regularization
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='sigmoid')  # Sigmoid activation for multilabel classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=100, batch_size=64, 
                    validation_split=0.2, callbacks=[early_stopping], verbose=0)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy*100:.2f}%')

# Generate predictions
predicted_probs = model.predict(X_test)
predicted_labels = (predicted_probs > 0.5).astype(int)

# Generate classification report
report = classification_report(y_test, predicted_labels, target_names=mlb.classes_)

print("Classification Report:\n", report)


163/163 [==============================] - 0s 1ms/step - loss: 0.0373 - accuracy: 0.4300
Test accuracy: 43.00%
163/163 [==============================] - 0s 1ms/step
Classification Report:
                                     precision    recall  f1-score   support

                 ACCOUNTS/EARNINGS       0.93      0.86      0.89        91
                   ASSET_TRANSFERS       0.60      0.28      0.39       113
               BALANCE_OF_PAYMENTS       0.87      0.66      0.75       109
BIOGRAPHIES,_PERSONALITIES,_PEOPLE       0.84      0.63      0.72        91
                 BONDS/DEBT_ISSUES       0.93      0.81      0.86       110
               CAPACITY/FACILITIES       0.74      0.63      0.68       412
                 COMMENT/FORECASTS       0.90      0.63      0.74       100
                 COMMODITY_MARKETS       0.91      0.95      0.93       290
                  CONSUMER_FINANCE       0.96      0.95      0.95        94
                   CONSUMER_PRICES       0.79    

C:\Users\gayat\.conda\envs\pracenv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
print(f'Test accuracy: {test_accuracy*100:.2f}%')

Test accuracy: 43.00%


## Model

In [30]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Assuming Dq is your features and mlb_df is your encoded labels
X = Dq
y = mlb_df.values

# Define the number of folds for cross-validation
n_splits = 10
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Placeholder for aggregated test set results
y_test_all = []
predicted_labels_all = []

for train_index, test_index in kf.split(X, y.argmax(1)):  # Using argmax to simulate stratification
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    num_classes = y_train.shape[1]
    
    # Define the model architecture
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='sigmoid')  # Sigmoid activation for multilabel classification
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Train the model with early stopping
    model.fit(X_train, y_train, epochs=100, batch_size=64, 
              validation_split=0.2, callbacks=[early_stopping], verbose=0)
    
    # Generate predictions for this fold's test set
    predicted_probs = model.predict(X_test)
    predicted_labels = (predicted_probs > 0.5).astype(int)
    
    # Aggregate the results
    y_test_all.append(y_test)
    predicted_labels_all.append(predicted_labels)

# Concatenate all test set results
y_test_all = np.concatenate(y_test_all, axis=0)
predicted_labels_all = np.concatenate(predicted_labels_all, axis=0)

# Generate and print the overall classification report
report = classification_report(y_test_all, predicted_labels_all, target_names=mlb.classes_)
print("Overall Classification Report:\n", report)


82/82 [==============================] - 0s 1ms/step
Overall Classification Report:
                                     precision    recall  f1-score   support

                 ACCOUNTS/EARNINGS       0.94      0.52      0.67       500
                   ASSET_TRANSFERS       0.00      0.00      0.00       500
               BALANCE_OF_PAYMENTS       0.80      0.04      0.08       500
BIOGRAPHIES,_PERSONALITIES,_PEOPLE       0.75      0.01      0.01       500
                 BONDS/DEBT_ISSUES       0.94      0.50      0.66       500
               CAPACITY/FACILITIES       0.67      0.53      0.60      2000
                 COMMENT/FORECASTS       0.00      0.00      0.00       500
                 COMMODITY_MARKETS       0.92      0.79      0.85      1500
                  CONSUMER_FINANCE       1.00      0.08      0.15       500
                   CONSUMER_PRICES       0.47      0.13      0.20       500
                  CONTRACTS/ORDERS       0.00      0.00      0.00       500
  

C:\Users\gayat\.conda\envs\pracenv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gayat\.conda\envs\pracenv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Assuming Dq is your features and mlb_df is your encoded labels
X = Dq
y = mlb_df.values

# Define the number of folds for cross-validation
n_splits = 10
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Placeholder for aggregated test set results
y_test_all = []
predicted_labels_all = []
accuracies = []

for train_index, test_index in kf.split(X, y.argmax(1)):  # Using argmax to simulate stratification
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    num_classes = y_train.shape[1]
    
    # Define the model architecture
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='sigmoid')  # Sigmoid activation for multilabel classification
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Train the model with early stopping
    model.fit(X_train, y_train, epochs=100, batch_size=64, 
              validation_split=0.2, callbacks=[early_stopping], verbose=0)
    
    # Generate predictions for this fold's test set
    predicted_probs = model.predict(X_test)
    predicted_labels = (predicted_probs > 0.5).astype(int)
    
    # Calculate accuracy for this fold
    accuracy = accuracy_score(y_test, predicted_labels)
    accuracies.append(accuracy)
    
    # Aggregate the results
    y_test_all.append(y_test)
    predicted_labels_all.append(predicted_labels)

# Concatenate all test set results
y_test_all = np.concatenate(y_test_all, axis=0)
predicted_labels_all = np.concatenate(predicted_labels_all, axis=0)

# Generate and print the overall classification report
report = classification_report(y_test_all, predicted_labels_all, target_names=mlb.classes_)
print("Overall Classification Report:\n", report)

# Calculate and print the average accuracy
avg_accuracy = np.mean(accuracies)
print("Average Accuracy:", avg_accuracy)


82/82 [==============================] - 0s 1ms/step
Overall Classification Report:
                                     precision    recall  f1-score   support

                 ACCOUNTS/EARNINGS       0.96      0.49      0.65       500
                   ASSET_TRANSFERS       0.00      0.00      0.00       500
               BALANCE_OF_PAYMENTS       1.00      0.01      0.02       500
BIOGRAPHIES,_PERSONALITIES,_PEOPLE       0.00      0.00      0.00       500
                 BONDS/DEBT_ISSUES       0.93      0.49      0.65       500
               CAPACITY/FACILITIES       0.68      0.54      0.60      2000
                 COMMENT/FORECASTS       0.00      0.00      0.00       500
                 COMMODITY_MARKETS       0.92      0.78      0.84      1500
                  CONSUMER_FINANCE       1.00      0.03      0.06       500
                   CONSUMER_PRICES       0.47      0.14      0.22       500
                  CONTRACTS/ORDERS       0.00      0.00      0.00       500
  

C:\Users\gayat\.conda\envs\pracenv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gayat\.conda\envs\pracenv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np

# Assuming Dq is your features and mlb_df is your encoded labels
X = Dq
y = mlb_df.values

# Define the number of folds for cross-validation
n_splits = 10
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Placeholder for aggregated test set results
y_test_all = []
predicted_labels_all = []
f1_macro = []
f1_micro = []
f1_weighted = []

for train_index, test_index in kf.split(X, y.argmax(1)):  # Using argmax to simulate stratification
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    num_classes = y_train.shape[1]
    
    # Define the model architecture
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='sigmoid')  # Sigmoid activation for multilabel classification
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Train the model with early stopping
    model.fit(X_train, y_train, epochs=100, batch_size=64, 
              validation_split=0.2, callbacks=[early_stopping], verbose=0)
    
    # Generate predictions for this fold's test set
    predicted_probs = model.predict(X_test)
    predicted_labels = (predicted_probs > 0.5).astype(int)
    
    # Calculate F1 scores for this fold
    f1_macro.append(f1_score(y_test, predicted_labels, average='macro'))
    f1_micro.append(f1_score(y_test, predicted_labels, average='micro'))
    f1_weighted.append(f1_score(y_test, predicted_labels, average='weighted'))
    
    # Aggregate the results
    y_test_all.append(y_test)
    predicted_labels_all.append(predicted_labels)

# Concatenate all test set results
y_test_all = np.concatenate(y_test_all, axis=0)
predicted_labels_all = np.concatenate(predicted_labels_all, axis=0)

# Generate and print the overall classification report
report = classification_report(y_test_all, predicted_labels_all, target_names=mlb.classes_)
print("Overall Classification Report:\n", report)

# Calculate and print the average F1 scores
avg_f1_macro = np.mean(f1_macro)
avg_f1_micro = np.mean(f1_micro)
avg_f1_weighted = np.mean(f1_weighted)

print("Macro-average F1 score:", avg_f1_macro)
print("Micro-average F1 score:", avg_f1_micro)
print("Weighted-average F1 score:", avg_f1_weighted)


82/82 [==============================] - 0s 1ms/step
Overall Classification Report:
                                     precision    recall  f1-score   support

                 ACCOUNTS/EARNINGS       0.92      0.52      0.66       500
                   ASSET_TRANSFERS       0.00      0.00      0.00       500
               BALANCE_OF_PAYMENTS       0.70      0.03      0.06       500
BIOGRAPHIES,_PERSONALITIES,_PEOPLE       1.00      0.01      0.02       500
                 BONDS/DEBT_ISSUES       0.94      0.50      0.65       500
               CAPACITY/FACILITIES       0.66      0.57      0.61      2000
                 COMMENT/FORECASTS       0.00      0.00      0.00       500
                 COMMODITY_MARKETS       0.92      0.79      0.85      1500
                  CONSUMER_FINANCE       1.00      0.04      0.08       500
                   CONSUMER_PRICES       0.53      0.22      0.31       500
                  CONTRACTS/ORDERS       0.00      0.00      0.00       500
  

C:\Users\gayat\.conda\envs\pracenv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gayat\.conda\envs\pracenv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, f1_score
import numpy as np

X = Dq  # Your features
y = mlb_df.values  # Your labels

n_splits = 10
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store f1 scores for each fold
f1_scores_macro = []
f1_scores_micro = []
f1_scores_weighted = []

for train_index, test_index in kf.split(X, y.argmax(1)):  # Assuming you have a single-label representation for stratification
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    num_classes = y_train.shape[1]
    
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.25),
        Dense(256, activation='relu'),
        Dropout(0.25),
        Dense(num_classes, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping], verbose=0)
    
    predicted_probs = model.predict(X_test)
    predicted_labels = (predicted_probs > 0.5).astype(int)
    
    # Calculate F1 scores for this fold
    f1_macro = f1_score(y_test, predicted_labels, average='macro')
    f1_micro = f1_score(y_test, predicted_labels, average='micro')
    f1_weighted = f1_score(y_test, predicted_labels, average='weighted')
    
    # Append scores of this fold to lists
    f1_scores_macro.append(f1_macro)
    f1_scores_micro.append(f1_micro)
    f1_scores_weighted.append(f1_weighted)

# Calculate average F1 scores across all folds
avg_f1_macro = np.mean(f1_scores_macro)
avg_f1_micro = np.mean(f1_scores_micro)
avg_f1_weighted = np.mean(f1_scores_weighted)

print(f"Average Macro-average F1 score: {avg_f1_macro}")
print(f"Average Micro-average F1 score: {avg_f1_micro}")
print(f"Average Weighted-average F1 score: {avg_f1_weighted}")


82/82 [==============================] - 0s 2ms/step
Average Macro-average F1 score: 0.45593744083651416
Average Micro-average F1 score: 0.6849665887318219
Average Weighted-average F1 score: 0.6128595571431876


### for 0.5 dropout 

Average Macro-average F1 score: 0.35566323207273537

Average Micro-average F1 score: 0.6443847448144114

Average Weighted-average F1 score: 0.5527350882776797

In [36]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, f1_score
import numpy as np

X = Dq  # Your features
y = mlb_df.values  # Your labels

n_splits = 10
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store f1 scores for each fold
f1_scores_macro = []
f1_scores_micro = []
f1_scores_weighted = []

for train_index, test_index in kf.split(X, y.argmax(1)):  # Assuming you have a single-label representation for stratification
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    num_classes = y_train.shape[1]
    
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.35),
        Dense(256, activation='relu'),
        Dropout(0.35),
        Dense(num_classes, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping], verbose=0)
    
    predicted_probs = model.predict(X_test)
    predicted_labels = (predicted_probs > 0.5).astype(int)
    
    # Calculate F1 scores for this fold
    f1_macro = f1_score(y_test, predicted_labels, average='macro')
    f1_micro = f1_score(y_test, predicted_labels, average='micro')
    f1_weighted = f1_score(y_test, predicted_labels, average='weighted')
    
    # Append scores of this fold to lists
    f1_scores_macro.append(f1_macro)
    f1_scores_micro.append(f1_micro)
    f1_scores_weighted.append(f1_weighted)

# Calculate average F1 scores across all folds
avg_f1_macro = np.mean(f1_scores_macro)
avg_f1_micro = np.mean(f1_scores_micro)
avg_f1_weighted = np.mean(f1_scores_weighted)

print(f"Average Macro-average F1 score: {avg_f1_macro}")
print(f"Average Micro-average F1 score: {avg_f1_micro}")
print(f"Average Weighted-average F1 score: {avg_f1_weighted}")


82/82 [==============================] - 0s 2ms/step
Average Macro-average F1 score: 0.4232924726590065
Average Micro-average F1 score: 0.6715908183143133
Average Weighted-average F1 score: 0.5919499534249267


In [ ]:
Average Macro-average F1 score: 0.45593744083651416
Average Micro-average F1 score: 0.6849665887318219
Average Weighted-average F1 score: 0.6128595571431876

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras import regularizers  # Import regularizers module

# Initialize variables to store true labels and predicted labels
true_labels = []
predicted_labels = []

num_folds = 10
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize list to store test accuracies
test_accuracies = []

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(skf.split(Dq, df['encoded_topics'])):
    print(f'Fold {fold + 1}/{num_folds}')

    # Split data into train and test sets for this fold
    X_train, X_test = Dq[train_index], Dq[test_index]
    y_train, y_test = df['encoded_topics'].iloc[train_index], df['encoded_topics'].iloc[test_index]

    num_classes = len(np.unique(np.concatenate((y_train, y_test))))

    y_train_encoded = to_categorical(y_train, num_classes=num_classes)
    y_test_encoded = to_categorical(y_test, num_classes=num_classes)

    X_train_flattened = X_train.reshape((X_train.shape[0], -1))
    X_test_flattened = X_test.reshape((X_test.shape[0], -1))

    # Define the model architecture with L1 regularization
    model = Sequential([
        Dense(512, activation='relu', kernel_regularizer=regularizers.l1(0.0001), input_shape=(X_train_flattened.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu', kernel_regularizer=regularizers.l1(0.0001)),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(X_train_flattened, y_train_encoded, epochs=100, batch_size=64, 
                        validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # Evaluate the model on the test set and collect predictions
    test_loss, test_accuracy = model.evaluate(X_test_flattened, y_test_encoded)
    print(f'Test accuracy for fold {fold + 1}: {test_accuracy*100:.2f}%')

    # Store test accuracy for this fold
    test_accuracies.append(test_accuracy)

    # Collect true labels and predicted labels
    true_labels.extend(y_test)
    predicted_probs = model.predict(X_test_flattened)
    predicted_labels.extend(np.argmax(predicted_probs, axis=1))

# Calculate average test accuracy
avg_test_accuracy = np.mean(test_accuracies)

# Generate classification report
report = classification_report(true_labels, predicted_labels)

# Add average accuracy to the report
report_with_avg_accuracy = f'Average Test Accuracy: {avg_test_accuracy*100:.2f}%\n\n{report}'
print("Classification Report with Average Test Accuracy:\n", report_with_avg_accuracy)


In [37]:
sampled_data.to_csv(r'D:\Capstone\Datasets\Data_preparation\sampled_data.csv', index=False)

In [ ]:

top_52_topics = df['topics'].value_counts().head(52)

# Create an empty DataFrame to store sampled data
sampled_data = pd.DataFrame(columns=df.columns)

for topic, count in top_52_topics.items():
    topic_data = df[df['topics'] == topic]
    
    # If there are more than 500 samples, randomly select 500
    if count > 500:
        topic_data = topic_data.sample(n=500, random_state=42)
    
    sampled_data = pd.concat([sampled_data, topic_data])

sampled_data.reset_index(drop=True, inplace=True)

sampled_data.shape
